In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

### YOLOv8을 사용한 얼굴 부위 분류 모델 학습

In [ ]:
import os
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
from PIL import Image
import shutil
import ast

In [ ]:
# YAML 파일 생성 함수
def create_yolo_yaml(base_path):
    yaml_content = f"""
path: {base_path}
train: Training/images
val: Validation/images
nc: 9
names: ['forehead', 'glabella', 'left_eye', 'right_eye', 'left_cheek', 'right_cheek', 'nose', 'mouth', 'chin']

train_labels: Training/labels
val_labels: Validation/labels
"""
    yaml_path = os.path.join(base_path, 'dataset.yaml')
    with open(yaml_path, 'w', encoding='utf-8') as f:
        f.write(yaml_content)
    return yaml_path

In [ ]:
# 손상된 이미지와 라벨링 같이 제거
def remove_corrupted_images(folder_path):
    corrupted_images = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(folder_path, filename)
            try:
                with Image.open(file_path) as img:
                    img.verify()
            except Exception as e:
                print(f"손상된 이미지 파일 발견: {filename}")
                corrupted_images.append(filename)
                os.remove(file_path)
    return corrupted_images

In [ ]:
# yolo형식으로 데이터 전처리 함수
def prepare_yolo_data(df, base_path):
    train_image_path = os.path.join(base_path, 'Training', 'images')
    val_image_path = os.path.join(base_path, 'Validation', 'images')
    train_label_path = os.path.join(base_path, 'Training', 'labels')
    val_label_path = os.path.join(base_path, 'Validation', 'labels')

    # 모든 필요한 폴더가 이미 존재하는지 확인
    if all(os.path.exists(path) for path in [train_image_path, val_image_path, train_label_path, val_label_path]):
        print("기존 데이터 폴더를 사용합니다.")
        return train_image_path, val_image_path, train_label_path, val_label_path

    # 폴더가 없는 경우 새로 생성
    os.makedirs(train_image_path, exist_ok=True)
    os.makedirs(val_image_path, exist_ok=True)
    os.makedirs(train_label_path, exist_ok=True)
    os.makedirs(val_label_path, exist_ok=True)

    print("Training 데이터에서 손상된 이미지 제거 중...")
    remove_corrupted_images(train_image_path)
    print("Validation 데이터에서 손상된 이미지 제거 중...")
    remove_corrupted_images(val_image_path)

    processed_count = 0
    for index, row in df.iterrows():
        try:
            info = row['info']
            images = row['images']

            filename = info['filename']
            facepart = images['facepart']
            bbox = images['bbox']
            width = images['width']
            height = images['height']

            src_img_path = os.path.join(base_path, row['split'], '01.원천데이터', filename)
            dst_img_path = os.path.join(train_image_path if 'Training' in row['split'] else val_image_path, filename)

            if not os.path.exists(src_img_path):
                print(f"{index}번 행을 건너뜁니다. 이미지 파일이 존재하지 않습니다: {filename}")
                continue

            # 이미지 파일 복사
            shutil.copy2(src_img_path, dst_img_path)

            if not all([filename, facepart is not None, width, height, bbox]):
                print(f"{index}번 행을 건너뜁니다. 파일에 대한 필수 정보가 누락되었습니다: {filename}")
                continue

            if (bbox == ['None', 'None', 'None', 'None']) or not all(isinstance(b, (int, float)) and b is not None for b in bbox):
                print(f"{index}번 행을 건너뜁니다. 유효하지 않은 bbox 값입니다. 파일: {filename}")
                continue

            bbox = [float(b) for b in bbox]
            width = float(width)
            height = float(height)

            x_center = (bbox[0] + bbox[2]) / 2 / width
            y_center = (bbox[1] + bbox[3]) / 2 / height
            bbox_width = (bbox[2] - bbox[0]) / width
            bbox_height = (bbox[3] - bbox[1]) / height

            yolo_format = f"{facepart} {x_center} {y_center} {bbox_width} {bbox_height}\n"

            label_path = train_label_path if 'Training' in row['split'] else val_label_path
            label_filename = os.path.splitext(filename)[0] + '.txt'

            with open(os.path.join(label_path, label_filename), 'a') as f:
                f.write(yolo_format)

            processed_count += 1

        except Exception as e:
            print(f"{index}번 행 처리 중 오류 발생: {str(e)}")
            print(f"행 데이터: {row}")

    print(f"총 {processed_count}개의 이미지를 처리했습니다.")
    return train_image_path, val_image_path, train_label_path, val_label_path

In [ ]:
# YOLO 모델 학습
def train_yolo_model(df, base_path):
    train_image_path, val_image_path, train_label_path, val_label_path = prepare_yolo_data(df, base_path)

    yaml_path = create_yolo_yaml(base_path)

    model = YOLO('yolov8n.yaml')
    results = model.train(data=yaml_path, epochs=10, imgsz=640)

    model.save('yolov8_facepart_model.pt')

    return model, results

In [ ]:
# 함수 실행
if __name__ == "__main__":
    base_path = os.path.abspath('/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터')

    # 저장한 데이터 프레임 불러오기
    df = pd.read_csv(base_path + '/json to df.csv')

    # csv로 불러오니 딕셔너리가 문자열로 다 바뀌어서 나와 딕셔너리로 변환
    for col in ['info', 'images', 'annotations', 'equipment']:
        df[col] = df[col].apply(lambda x: ast.literal_eval(x) if x else None)

    # YOLO 모델 학습
    trained_model, training_results = train_yolo_model(df, base_path)

    if trained_model:
        print("모델 학습이 완료되었습니다.")
        print("학습 결과:", training_results)
        print("학습된 모델이 'yolov8_facepart_model.pt'로 저장되었습니다.")
    else:
        print("모델 학습에 실패했습니다.")